# Dog Breeds

In [1]:
# import useful packages
import pandas as pd
import numpy as np
from numpy import argmax
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import History, TensorBoard
from keras import applications

import tensorflow as tf

Using TensorFlow backend.


In [2]:
# callbacks
history = History()
tb = TensorBoard()

In [3]:
labels_df = pd.read_csv('/mnt/DataDisk/jodahr/data/Dogs/labels.csv')

In [4]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
id       10222 non-null object
breed    10222 non-null object
dtypes: object(2)
memory usage: 159.8+ KB


In [5]:
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


## Prepare Training Data

In [6]:
import sys, os, subprocess
from shutil import copyfile
import shutil

In [7]:
# make subdirs for dog breeds
for breed in breeds:
    os.makedirs('/mnt/DataDisk/jodahr/data/Dogs/training/' + breed, exist_ok=True)
    os.makedirs('/mnt/DataDisk/jodahr/data/Dogs/validation/' + breed, exist_ok=True)

NameError: name 'breeds' is not defined

In [ ]:
# image path
path = '/mnt/DataDisk/jodahr/data/Dogs/train/'

In [ ]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(labels_df, test_size=0.3)

In [134]:
for index, row in train.iterrows():
    filename = row['id'] + '.jpg'
    src = path + filename
    target = '/mnt/DataDisk/jodahr/data/Dogs/training/' + row['breed']
    shutil.copy(src=src, dst=target)

In [135]:
for index, row in test.iterrows():
    filename = row['id'] + '.jpg'
    src = path + filename
    target = '/mnt/DataDisk/jodahr/data/Dogs/validation/' + row['breed']
    shutil.copy(src=src, dst=target)

## Keras Model

In [19]:
# dimensions of our images.
img_width, img_height = 250, 250

train_data_dir = '/mnt/DataDisk/jodahr/data/Dogs/training/'
validation_data_dir = '/mnt/DataDisk/jodahr/data/Dogs/validation/'
nb_train_samples = 7155
nb_validation_samples = 3067
epochs = 150
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [9]:
base_model = applications.VGG16(include_top=False, input_shape=input_shape)

In [10]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250, 250, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0         
__________

In [11]:
for layer in base_model.layers[:]:
    layer.trainable = False

In [12]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250, 250, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0         
__________

In [13]:
# model topology
top_model = Sequential()

top_model.add(Conv2D(32, (3, 3), input_shape=base_model.output_shape[1:]))
top_model.add(Activation('relu'))
top_model.add(MaxPooling2D(pool_size=(2, 2)))

top_model.add(Flatten())
top_model.add(Dense(32))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(120))
top_model.add(Activation('softmax'))



In [22]:
from keras import Model
from keras import metrics

model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[metrics.categorical_accuracy])

In [23]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [24]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 7155 images belonging to 120 classes.
Found 3067 images belonging to 120 classes.


In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250, 250, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0         
__________

In [29]:
with tf.device('/gpu:0'):
    model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [history, tb])

Epoch 1/150
111/111 [==============================] - 108s 971ms/step - loss: 4.7443 - categorical_accuracy: 0.0145 - val_loss: 4.6810 - val_categorical_accuracy: 0.0173
Epoch 2/150
111/111 [==============================] - 105s 942ms/step - loss: 4.6323 - categorical_accuracy: 0.0189 - val_loss: 4.5422 - val_categorical_accuracy: 0.0209
Epoch 3/150
111/111 [==============================] - 104s 941ms/step - loss: 4.5282 - categorical_accuracy: 0.0214 - val_loss: 4.4599 - val_categorical_accuracy: 0.0303
Epoch 4/150
111/111 [==============================] - 105s 942ms/step - loss: 4.4729 - categorical_accuracy: 0.0232 - val_loss: 4.4349 - val_categorical_accuracy: 0.0283
Epoch 5/150
111/111 [==============================] - 105s 943ms/step - loss: 4.4177 - categorical_accuracy: 0.0228 - val_loss: 4.3812 - val_categorical_accuracy: 0.0336
Epoch 6/150
111/111 [==============================] - 104s 940ms/step - loss: 4.3759 - categorical_accuracy: 0.0294 - val_loss: 4.3464 - val_cat

Epoch 49/150
111/111 [==============================] - 104s 941ms/step - loss: 3.8291 - categorical_accuracy: 0.0797 - val_loss: 3.8522 - val_categorical_accuracy: 0.0768
Epoch 50/150
111/111 [==============================] - 104s 941ms/step - loss: 3.8262 - categorical_accuracy: 0.0783 - val_loss: 3.8481 - val_categorical_accuracy: 0.0795
Epoch 51/150
111/111 [==============================] - 104s 939ms/step - loss: 3.8266 - categorical_accuracy: 0.0765 - val_loss: 3.8737 - val_categorical_accuracy: 0.0695
Epoch 52/150
111/111 [==============================] - 105s 942ms/step - loss: 3.8382 - categorical_accuracy: 0.0753 - val_loss: 3.8590 - val_categorical_accuracy: 0.0758
Epoch 53/150
111/111 [==============================] - 104s 940ms/step - loss: 3.8203 - categorical_accuracy: 0.0814 - val_loss: 3.8514 - val_categorical_accuracy: 0.0795
Epoch 54/150
111/111 [==============================] - 105s 942ms/step - loss: 3.8095 - categorical_accuracy: 0.0715 - val_loss: 3.8479 - v

111/111 [==============================] - 105s 942ms/step - loss: 3.7706 - categorical_accuracy: 0.0844 - val_loss: 3.8382 - val_categorical_accuracy: 0.0738
Epoch 97/150
111/111 [==============================] - 104s 941ms/step - loss: 3.7636 - categorical_accuracy: 0.0874 - val_loss: 3.8302 - val_categorical_accuracy: 0.0758
Epoch 98/150
111/111 [==============================] - 104s 940ms/step - loss: 3.7414 - categorical_accuracy: 0.0866 - val_loss: 3.8275 - val_categorical_accuracy: 0.0798
Epoch 99/150
111/111 [==============================] - 104s 940ms/step - loss: 3.7491 - categorical_accuracy: 0.0815 - val_loss: 3.8312 - val_categorical_accuracy: 0.0791
Epoch 100/150
111/111 [==============================] - 105s 942ms/step - loss: 3.7576 - categorical_accuracy: 0.0820 - val_loss: 3.8294 - val_categorical_accuracy: 0.0838
Epoch 101/150
111/111 [==============================] - 104s 940ms/step - loss: 3.7330 - categorical_accuracy: 0.0858 - val_loss: 3.8440 - val_categori

111/111 [==============================] - 105s 942ms/step - loss: 3.7207 - categorical_accuracy: 0.0866 - val_loss: 3.8810 - val_categorical_accuracy: 0.0778
Epoch 144/150
111/111 [==============================] - 104s 939ms/step - loss: 3.7332 - categorical_accuracy: 0.0843 - val_loss: 3.8446 - val_categorical_accuracy: 0.0834
Epoch 145/150
111/111 [==============================] - 105s 943ms/step - loss: 3.7224 - categorical_accuracy: 0.0884 - val_loss: 3.8438 - val_categorical_accuracy: 0.0811
Epoch 146/150
111/111 [==============================] - 105s 942ms/step - loss: 3.7261 - categorical_accuracy: 0.0883 - val_loss: 3.8372 - val_categorical_accuracy: 0.0775
Epoch 147/150
111/111 [==============================] - 104s 941ms/step - loss: 3.7083 - categorical_accuracy: 0.0852 - val_loss: 3.8489 - val_categorical_accuracy: 0.0814
Epoch 148/150
111/111 [==============================] - 104s 940ms/step - loss: 3.7206 - categorical_accuracy: 0.0922 - val_loss: 3.8679 - val_categ

In [30]:
model.save_weights('my_model_weights.h5')

In [31]:
## custom batch generator
test_path = '/mnt/DataDisk/jodahr/data/Dogs/test/'

In [48]:
import cv2

In [274]:
# not very elegant, but works
def customGenerator(path, batch_size=32, target_size=(250,250), add_names=False):
    '''Generator wich returns batches of img arrays. Alternatively you can add the filenames.'''
    file_names = os.listdir(test_path)  # list of filenames
    chunks = [file_names[x:x+batch_size] for x in range(0, len(file_names), batch_size)]  # divided into chunks
    
    for chunk in chunks:
        full_names = [test_path + '/' + file_name for file_name in chunk]  # create full filenames
        img_batch = [load_img(full_name, target_size=target_size) for full_name in full_names]  # load image batches
        arr_batch = np.empty(shape=(1,target_size[0],target_size[1],3))  # empty multidimensional numpy array
        #arr_batch = []
        # rescale arrays and combine them
        for element in img_batch:
            img_arr = np.expand_dims(img_to_array(element)/255., axis=0)
            arr_batch = np.concatenate((arr_batch, img_arr), axis=0)
        # yield
        if add_names==False:
            yield arr_batch[1:,:,:,:]
        else:
            yield (arr_batch[1:,:,:,:], chunk)

In [292]:
gen = customGenerator(test_path, add_names=True)

In [291]:
next(gen)

(array([[[[ 0.84705883,  0.76078433,  0.67843139],
          [ 0.80392158,  0.5529412 ,  0.3019608 ],
          [ 0.63921571,  0.3882353 ,  0.13725491],
          ..., 
          [ 0.11764706,  0.05098039,  0.01960784],
          [ 0.10980392,  0.05490196,  0.01960784],
          [ 0.10980392,  0.05490196,  0.01960784]],
 
         [[ 1.        ,  0.98039216,  0.89803922],
          [ 0.78431374,  0.54901963,  0.30588236],
          [ 0.78823531,  0.54901963,  0.3019608 ],
          ..., 
          [ 0.11764706,  0.05098039,  0.01960784],
          [ 0.10980392,  0.05490196,  0.01960784],
          [ 0.10980392,  0.05490196,  0.01960784]],
 
         [[ 1.        ,  0.92156863,  0.8392157 ],
          [ 0.82352942,  0.59215689,  0.36470589],
          [ 0.7647059 ,  0.52941179,  0.28627452],
          ..., 
          [ 0.11764706,  0.05098039,  0.01960784],
          [ 0.10980392,  0.05490196,  0.01960784],
          [ 0.10980392,  0.05490196,  0.01960784]],
 
         ..., 
         [

In [198]:
class_names = [cls for cls in train_generator.class_indices]

In [197]:
pre

['affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle',
 'bedlington_terrier',
 'bernese_mountain_dog',
 'black-and-tan_coonhound',
 'blenheim_spaniel',
 'bloodhound',
 'bluetick',
 'border_collie',
 'border_terrier',
 'borzoi',
 'boston_bull',
 'bouvier_des_flandres',
 'boxer',
 'brabancon_griffon',
 'briard',
 'brittany_spaniel',
 'bull_mastiff',
 'cairn',
 'cardigan',
 'chesapeake_bay_retriever',
 'chihuahua',
 'chow',
 'clumber',
 'cocker_spaniel',
 'collie',
 'curly-coated_retriever',
 'dandie_dinmont',
 'dhole',
 'dingo',
 'doberman',
 'english_foxhound',
 'english_setter',
 'english_springer',
 'entlebucher',
 'eskimo_dog',
 'flat-coated_retriever',
 'french_bulldog',
 'german_shepherd',
 'german_short-haired_pointer',
 'giant_schnauzer',
 'golden_retriever',
 'gordon_setter',
 'great_dane',
 'great_pyrenees',
 'greater_swiss_mountain_dog',
 'groenendael',


In [200]:
predictions = model.predict(next(gen))

In [ ]:
for 

In [276]:
predictions

array([[  8.66719801e-03,   1.36663876e-02,   9.29446614e-05, ...,
          1.34669526e-05,   4.20302246e-03,   3.58942925e-04],
       [  1.90508254e-02,   2.92434283e-02,   1.61392862e-04, ...,
          3.28782335e-07,   1.53485574e-02,   2.59571169e-02],
       [  2.83657664e-05,   1.34259956e-02,   1.61422722e-05, ...,
          2.22239745e-04,   2.10064813e-03,   1.31909736e-04],
       ..., 
       [  1.85186355e-09,   1.73129934e-06,   1.91986590e-04, ...,
          4.66543213e-02,   3.26589688e-05,   4.03094758e-09],
       [  1.64215191e-04,   6.60411939e-02,   5.29717654e-05, ...,
          2.54850274e-05,   8.80080368e-03,   3.71290557e-03],
       [  1.73287913e-02,   1.78360716e-02,   8.16231873e-03, ...,
          1.24997168e-03,   1.16306208e-02,   1.38936155e-02]], dtype=float32)

In [217]:
import pandas as pd

In [313]:
def get_submission(my_generator_tuple, model, class_names):
    result = pd.DataFrame()
    predictions = []
    names = []
    cols = ['id'] + class_names
    for element in my_generator_tuple:
        predictions = model.predict(element[0])
        names = element[1]
        temp = pd.DataFrame(predictions)
        temp.columns = class_names
        temp['names'] = names
        result = pd.concat([result, temp])
    #result.columns = class_names
    #result['names'] = names
    result['id'] = result.names.apply(lambda x: x.split('.jpg')[0])
    return result[cols]

In [314]:
gen = customGenerator(test_path, add_names=True)

In [315]:
df_sub = get_submission(gen, model, class_names)

In [316]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10357 entries, 0 to 20
Columns: 121 entries, id to yorkshire_terrier
dtypes: float32(120), object(1)
memory usage: 4.9+ MB


In [317]:
df_sub.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,2ce015d0d017c595bb64627a5749e3bd,1.157716e-02,4.835804e-03,0.027818,0.032993,0.007021,0.002110,6.514817e-03,0.006022,0.000925,...,8.090748e-03,0.002122,0.006315,0.000318,0.002031,0.000304,1.536332e-03,0.007216,0.007534,3.407072e-03
1,5bad43e776606caab0912c9e7f0e75ff,9.511627e-04,2.147182e-02,0.004653,0.005561,0.004169,0.010553,2.682158e-03,0.002632,0.009189,...,9.561103e-03,0.006851,0.001696,0.000436,0.001779,0.020222,7.231265e-03,0.004216,0.009269,6.476256e-03
2,89357a5cef0812f2ba888041de62a243,7.265832e-06,6.479005e-04,0.000827,0.003985,0.013822,0.000683,5.336166e-07,0.001787,0.010301,...,2.225210e-04,0.000009,0.070151,0.005030,0.066420,0.002201,1.981079e-07,0.041645,0.000604,4.121411e-06
3,b8da90f454f62fb33c83715a404aca3b,1.803837e-09,2.629415e-07,0.001661,0.000043,0.043727,0.023229,2.197491e-07,0.057545,0.023177,...,5.995201e-07,0.057286,0.008618,0.028716,0.008926,0.000309,3.000939e-08,0.025864,0.000128,3.689647e-08
4,000621fb3cbb32d8935728e48679680e,7.988258e-06,2.821555e-02,0.000588,0.001447,0.006184,0.007749,1.309027e-05,0.000572,0.024533,...,8.380160e-04,0.001081,0.004419,0.003700,0.008177,0.083709,7.171297e-05,0.006364,0.004820,3.975941e-04


In [318]:
df_sub.to_csv('test_20180325.csv', index=False)

In [307]:
df_sub['id'] = df_sub.names.apply(lambda x: x.split('.jpg')[0])
cols = ['id'] + class_names
df_sub[cols]

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,ffb3a78e4a162c173219194687eb6054,4.964123e-03,2.292066e-02,0.020423,0.024829,0.007781,7.997761e-03,4.293246e-03,4.821258e-03,0.009342,...,0.002738,3.733022e-03,9.822696e-03,2.508722e-03,6.091225e-03,0.009322,1.757284e-03,0.008865,0.011454,6.989223e-03
1,ffb55dbaa32939c109ef42df0668e077,3.641260e-06,1.329235e-05,0.011611,0.001259,0.028737,1.429782e-02,2.931678e-05,3.431739e-02,0.014508,...,0.000015,2.943945e-02,1.640671e-02,1.638699e-02,1.126600e-02,0.000388,1.912240e-06,0.026700,0.000794,4.937038e-06
2,ffb68ded4a2247b3c011840ece0a605c,1.765690e-04,1.115593e-02,0.000057,0.000465,0.000027,3.952796e-05,1.223583e-03,1.371287e-07,0.000026,...,0.025404,1.410393e-04,4.843663e-07,5.266557e-15,1.066823e-08,0.000569,1.352255e-02,0.000007,0.005035,4.440775e-03
3,ffb8bceeac5b3f587f67391ad346d454,7.390619e-05,1.649890e-05,0.032647,0.000936,0.004391,6.549921e-03,2.735183e-03,5.678755e-02,0.001180,...,0.000005,4.220213e-02,5.620310e-05,7.477730e-05,1.493562e-05,0.000082,4.612583e-04,0.001162,0.007072,1.800847e-04
4,ffbb6663b38e903dbd1efa5dce2d0bbb,7.302277e-06,2.201247e-02,0.000024,0.000182,0.000104,9.496404e-05,9.247853e-05,1.696455e-07,0.000312,...,0.011698,2.265531e-04,3.700886e-06,6.313635e-12,9.797548e-07,0.006858,2.832103e-03,0.000035,0.003735,1.196462e-03
5,ffbcda9eb84339cc5be15fd9900596a2,3.833268e-05,1.945516e-04,0.007154,0.005106,0.023243,5.988509e-03,2.341162e-05,1.271904e-02,0.015437,...,0.000212,1.074401e-03,4.326552e-02,1.679385e-02,3.652578e-02,0.001177,2.180068e-06,0.039987,0.001064,1.683983e-05
6,ffbcf032231bb0b025f9070d42ab7e8f,8.283211e-09,9.892879e-04,0.000084,0.000022,0.004493,9.863685e-02,7.200327e-07,4.649433e-03,0.034677,...,0.000005,1.663580e-02,8.847610e-05,3.564705e-03,1.160738e-04,0.057227,7.832055e-06,0.001436,0.002120,1.264341e-05
7,ffc0233904f4d03afe1484be791e09d6,2.613924e-02,1.129146e-02,0.014004,0.032468,0.001269,3.330973e-04,1.560937e-02,3.172185e-04,0.000124,...,0.016011,1.055968e-03,5.933307e-04,2.963574e-07,8.284627e-05,0.000292,8.990199e-03,0.000986,0.011440,1.261263e-02
8,ffd06687c72445b0c6e8a130a0a8711a,6.481498e-05,2.849161e-02,0.000125,0.000666,0.000328,2.642561e-04,4.624598e-04,1.848259e-06,0.000988,...,0.014121,5.202889e-04,2.934717e-05,3.126022e-09,1.559910e-05,0.010093,5.771452e-03,0.000164,0.005560,2.824786e-03
9,ffd304c521f43819f3824177fd9efeb0,2.801040e-07,9.948320e-06,0.002845,0.000696,0.028450,9.637496e-03,7.541211e-07,1.616295e-02,0.020645,...,0.000011,3.296572e-03,3.807002e-02,2.921159e-02,3.844613e-02,0.000785,5.946529e-08,0.039363,0.000238,5.109840e-07


In [306]:
cols

['id',
 'affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle',
 'bedlington_terrier',
 'bernese_mountain_dog',
 'black-and-tan_coonhound',
 'blenheim_spaniel',
 'bloodhound',
 'bluetick',
 'border_collie',
 'border_terrier',
 'borzoi',
 'boston_bull',
 'bouvier_des_flandres',
 'boxer',
 'brabancon_griffon',
 'briard',
 'brittany_spaniel',
 'bull_mastiff',
 'cairn',
 'cardigan',
 'chesapeake_bay_retriever',
 'chihuahua',
 'chow',
 'clumber',
 'cocker_spaniel',
 'collie',
 'curly-coated_retriever',
 'dandie_dinmont',
 'dhole',
 'dingo',
 'doberman',
 'english_foxhound',
 'english_setter',
 'english_springer',
 'entlebucher',
 'eskimo_dog',
 'flat-coated_retriever',
 'french_bulldog',
 'german_shepherd',
 'german_short-haired_pointer',
 'giant_schnauzer',
 'golden_retriever',
 'gordon_setter',
 'great_dane',
 'great_pyrenees',
 'greater_swiss_mountain_dog',
 'groenen

In [304]:
df_sub.head()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier,names,id
0,0.004964,0.022921,0.020423,0.024829,0.007781,0.007998,0.004293,4.821258e-03,0.009342,0.002844,...,9.822696e-03,2.508722e-03,6.091225e-03,0.009322,0.001757,0.008865,0.011454,0.006989,ffb3a78e4a162c173219194687eb6054.jpg,ffb3a78e4a162c173219194687eb6054
1,0.000004,0.000013,0.011611,0.001259,0.028737,0.014298,0.000029,3.431739e-02,0.014508,0.016231,...,1.640671e-02,1.638699e-02,1.126600e-02,0.000388,0.000002,0.026700,0.000794,0.000005,ffb55dbaa32939c109ef42df0668e077.jpg,ffb55dbaa32939c109ef42df0668e077
2,0.000177,0.011156,0.000057,0.000465,0.000027,0.000040,0.001224,1.371287e-07,0.000026,0.000005,...,4.843663e-07,5.266557e-15,1.066823e-08,0.000569,0.013523,0.000007,0.005035,0.004441,ffb68ded4a2247b3c011840ece0a605c.jpg,ffb68ded4a2247b3c011840ece0a605c
3,0.000074,0.000016,0.032647,0.000936,0.004391,0.006550,0.002735,5.678755e-02,0.001180,0.000239,...,5.620310e-05,7.477730e-05,1.493562e-05,0.000082,0.000461,0.001162,0.007072,0.000180,ffb8bceeac5b3f587f67391ad346d454.jpg,ffb8bceeac5b3f587f67391ad346d454
4,0.000007,0.022012,0.000024,0.000182,0.000104,0.000095,0.000092,1.696455e-07,0.000312,0.000086,...,3.700886e-06,6.313635e-12,9.797548e-07,0.006858,0.002832,0.000035,0.003735,0.001196,ffbb6663b38e903dbd1efa5dce2d0bbb.jpg,ffbb6663b38e903dbd1efa5dce2d0bbb


In [299]:
pd.DataFrame(model.predict(next(gen)[0]))

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,4.847685e-08,1.293495e-07,7.142829e-03,0.000033,8.609867e-03,1.189664e-02,2.711941e-05,1.384494e-01,3.221923e-03,3.892942e-04,...,4.669332e-08,1.027783e-01,4.033699e-05,4.792983e-04,1.774221e-05,2.442142e-05,3.153348e-06,1.543514e-03,0.001033,8.141250e-07
1,1.270168e-06,1.042112e-05,9.965837e-03,0.000599,3.265768e-02,1.740819e-02,4.520308e-05,5.746185e-02,1.581398e-02,1.357656e-02,...,2.129487e-05,4.592009e-02,8.247143e-03,1.302265e-02,5.583113e-03,4.345822e-04,7.502006e-06,2.364263e-02,0.001132,5.605015e-06
2,4.696525e-03,2.290840e-02,1.906859e-04,0.002687,1.359560e-06,3.734564e-06,1.914438e-02,1.455506e-08,5.899144e-06,5.602906e-07,...,2.024133e-02,1.517217e-04,2.456583e-08,1.728060e-15,2.117221e-09,2.862016e-04,4.158496e-02,9.256034e-07,0.014289,2.847234e-02
3,1.672391e-02,8.788795e-04,3.580221e-04,0.001527,1.576711e-09,8.381488e-08,1.848059e-01,2.877760e-09,8.251907e-10,7.075921e-12,...,7.077620e-05,6.268935e-05,1.492606e-13,1.874724e-21,1.818215e-15,2.585664e-07,4.931156e-02,2.472243e-10,0.027591,1.039180e-01
4,4.764122e-04,9.196560e-04,9.215328e-03,0.015434,1.899653e-02,2.100000e-03,1.530654e-04,1.171448e-02,5.772892e-03,8.138245e-03,...,3.809097e-03,3.204215e-04,4.150610e-02,3.392897e-03,2.128355e-02,3.612888e-04,3.555209e-05,4.320706e-02,0.002542,9.242539e-05
5,4.244670e-08,1.574225e-02,9.257778e-07,0.000078,1.706415e-04,1.476192e-06,3.161075e-08,6.366911e-09,5.563954e-04,3.961199e-04,...,1.367521e-03,1.461070e-07,7.066033e-05,3.124876e-10,5.857433e-05,6.009514e-03,1.837190e-06,1.467160e-04,0.000824,5.982155e-06
6,3.884103e-08,6.259249e-06,1.796884e-03,0.000253,3.264154e-02,1.829353e-02,2.618490e-07,1.807967e-02,2.769084e-02,5.127278e-02,...,2.616085e-06,1.214712e-02,2.653526e-02,3.870709e-02,3.361572e-02,1.510170e-03,3.460343e-08,3.371302e-02,0.000184,2.827911e-07
7,1.541406e-02,6.892887e-03,1.867199e-02,0.036760,4.779892e-03,8.756145e-04,4.974525e-03,3.414055e-03,4.038532e-04,8.314917e-04,...,1.193774e-02,5.591228e-04,4.800574e-03,1.041047e-04,1.213294e-03,1.456854e-04,1.554905e-03,5.186857e-03,0.007603,3.409334e-03
8,1.585345e-04,4.113255e-04,2.428345e-02,0.003841,1.841808e-02,1.300575e-02,1.604122e-03,4.083299e-02,1.048866e-02,5.435311e-03,...,3.816764e-04,2.838984e-02,4.783686e-03,4.692473e-03,3.138494e-03,1.272224e-03,7.073253e-04,1.477244e-02,0.006385,3.944759e-04
9,1.624531e-03,6.535420e-02,8.081776e-04,0.006500,1.562931e-04,1.632125e-04,3.609805e-03,3.097961e-06,1.176712e-03,1.401196e-04,...,1.295369e-02,4.185833e-04,5.100990e-05,4.777508e-08,3.996796e-05,7.475850e-03,6.999522e-03,2.149412e-04,0.015026,1.122399e-02


In [289]:
df_sub

,names


In [244]:
def get_results(my_generator_tuple, model, class_names):
    result = pd.DataFrame()
    for element in my_generator_tuple:
        predictions = model.predict(element[0])
        names = element[1]
        temp = pd.DataFrame({'idx': predictions.argmax(axis=1),
                             'score': predictions.max(axis=1),
                             'filenames': names})
        temp['class_name'] = temp['idx'].apply(lambda x: class_names[x])
        result = pd.concat([result,temp])
    return result
    

In [247]:
df = get_results(gen, model, class_names)

In [260]:
from shutil import copyfile

#copyfile(src, dst)

In [272]:
# copy result to folder
def make_result_folder(df, class_names, test_path):
        for element in class_names:
            os.makedirs('/mnt/DataDisk/jodahr/data/Dogs/result/' + element, exist_ok=True)
        for idx, row in df.iterrows():
            dest = '/mnt/DataDisk/jodahr/data/Dogs/result/'\
                + row['class_name']\
                + '/' + str(row['score'])\
                + '_' + row['filenames']
            src = test_path + '/' + row['filenames']
            copyfile(src,dest)

In [273]:
make_result_folder(df, class_names, test_path)

In [265]:
for idx, row in df.iterrows():
    print(row['filenames'])
    break

2ce015d0d017c595bb64627a5749e3bd.jpg


In [251]:
df.sort_values('score', ascending=False)

,filenames,idx,score,class_name
20,890f96dbc1f32ff2066a5572aaf78b9c.jpg,84,0.682099,papillon
24,bf29aa547c53fe893b2c3547d58063cd.jpg,42,0.644949,entlebucher
26,f1b2c118e65c95ba1a00d102787d19a6.jpg,9,0.642773,beagle
0,6d6519ebd4a475643e4c64043005cbb2.jpg,61,0.580584,japanese_spaniel
19,c636bf8d464b3601bf70681d1f1dd173.jpg,61,0.577773,japanese_spaniel
16,4423c704fafa60bf1df144f738340cef.jpg,86,0.567238,pembroke
9,e073f14cd5f4c1a62cdb0108cca62fdd.jpg,61,0.567231,japanese_spaniel
29,13b4ff9966c0eb2586caff2225cac832.jpg,13,0.549237,blenheim_spaniel
2,d8e36262aac18be8c6ff387d097d8e2b.jpg,13,0.544927,blenheim_spaniel
31,5b54ee9719a556eb94c6ce232f6d78ff.jpg,13,0.537397,blenheim_spaniel


In [215]:
def decode(predictions, class_names):
       highest_score_index = predictions.argmax(axis=1)
    highest_scores = predictions.max(axis=1)
    class_name = [cls for cls in predictions]
    return np.array([class_name, highest_scores])

In [216]:
decode(predictions, class_names)

NameError: name 'class_name' is not defined

In [210]:
predictions.argmax(axis=1)

array([ 97, 109,  52,  97, 109,  73,  31,  58,   9,  42,  54,  52,   2,
        90,  90,  90,  86,  90,   7,  73,  12,  61,   7, 102,  75, 101,
        87,  93, 101,  90,   1,  68])

In [144]:
test = customGenerator(test_path,add_names=True)

In [166]:
bla = next(test)

In [167]:
len(bla[0])

32

In [131]:
bla_test = test_path + '/' + bla[1][0]

In [132]:
img_to_array(load_img(bla_test))

array([[[ 232.,  224.,  221.],
        [ 216.,  194.,  173.],
        [ 187.,  142.,  100.],
        ..., 
        [  28.,   14.,    5.],
        [  28.,   14.,    5.],
        [  28.,   14.,    5.]],

       [[ 216.,  209.,  203.],
        [ 249.,  227.,  206.],
        [ 185.,  140.,   98.],
        ..., 
        [  28.,   14.,    5.],
        [  28.,   14.,    5.],
        [  28.,   14.,    5.]],

       [[ 168.,  159.,  154.],
        [ 255.,  250.,  229.],
        [ 204.,  161.,  119.],
        ..., 
        [  28.,   14.,    5.],
        [  28.,   14.,    5.],
        [  28.,   14.,    5.]],

       ..., 
       [[ 112.,   70.,   22.],
        [ 113.,   71.,   23.],
        [ 116.,   74.,   26.],
        ..., 
        [ 114.,   72.,   14.],
        [ 113.,   71.,   13.],
        [ 112.,   70.,   12.]],

       [[ 114.,   72.,   24.],
        [ 114.,   72.,   24.],
        [ 116.,   74.,   26.],
        ..., 
        [ 120.,   80.,   21.],
        [ 119.,   79.,   20.],
        [ 

In [126]:
bla_test

'/mnt/DataDisk/jodahr/data/Dogs/test//2ce015d0d017c595bb64627a5749e3bd.jpg'

In [38]:
test.next()

AttributeError: 'list_iterator' object has no attribute 'next'

In [ ]:
model.predict

In [28]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15482518694837723954, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 183631872
 locality {
   bus_id: 1
 }
 incarnation: 1626520279204157811
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:0c:00.0"]